This is the third attempt at Time Series Forecasting, this time with the goal to set up a live feed

I'll start by moving over my old code (and maybe making it prettier? I think we can skip over the plotting for now)

This notebook was written based on this tutorial
https://www.kaggle.com/code/someadityamandal/bitcoin-time-series-forecasting/notebook

Additional info to the last iteration:
https://www.youtube.com/watch?v=P_SIZDsI3Ro
https://www.youtube.com/watch?v=TF2Nx_ifmrU

We'll do plotting and whatnot towards the bottom. Those'll be their own sets of imports

In [ ]:
!pip install chart_studio

import numpy as np
import pandas as pd
import keras.layers as layers
import keras.models as models

import datetime, pytz
from sklearn.preprocessing import MinMaxScaler

import os
print(os.listdir('./data'))

The question is now, how do we take old data and new data into account?
I /think/ that the vibe is that we train a model on the historic data, then we feed the new data into it.
So we'll make the model, then feed new data into it, then plot.
That's the goal for tonight.